In [15]:
import pandas as pd
import pandas_ta as ta
import talib
import requests
import time
import datetime
import json
import yfinance as yf
import mplfinance as mpf
from io import StringIO
#from google.colab import data_table
#data_table.enable_dataframe_formatter()

In [ ]:
def get_stock_list():
  res = requests.get("https://isin.twse.com.tw/isin/class_main.jsp?owncode=&stockname=&isincode=&market=1&issuetype=1&industry_code=&Page=1&chklike=Y")
  df = pd.read_html(res.text)[0]        # Parse the Source Code into a Pandas DataFrame
  df = df.drop([0,1,4,5,8,9],axis = 1)  # Drop Useless Columns
  df.columns = df.iloc[0]               # Replace DataFrame Columns Title
  df = df.iloc[1:]
  return df

In [ ]:
def plot_stcok_k_chart(CLIENT_ID, stock="0050" , date_from='2020-01-01' ):
  """
  進行個股K線繪製，回傳至於雲端圖床的連結。將顯示包含5MA、20MA及量價關係，預設為'2020-01-01'迄今收盤價。
  :stock :個股代碼(字串)，預設0050。
  :date_from :起始日(字串)，格式為%Y-%m-%d，預設自2020-01-01起。
  """
  stock = str(stock)+".tw"
  # df = web.DataReader(stock, 'yahoo', date_from) # 已知有未修復的錯誤
  df = yf.download(stock, date_from)
  mpf.plot(df,type='candle',mav=(5,20),volume=True,title=stock.upper() ,savefig='testsave.png')
  PATH = "testsave.png"
  im = pyimgur.Imgur(CLIENT_ID)
  uploaded_image = im.upload_image(PATH, title=stock+" candlestick chart")
  return uploaded_image.link

# Main get stocker data

In [1]:
import requests
from io import StringIO
import pandas as pd
import pandas_ta as ta
import numpy as np
import datetime
import time
import sqlite3

class Stocker:
  def __init__(self, n_days, db_name):
    #self.name = name
    self.today = datetime.datetime.now()
    self.country = "TW"
    self.source = "TWSE"
    self.rank = ""
    self.data = None
    self.n_days = n_days
    self.db_name = db_name

  def crawl_price(self, date):
    r = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + str(date).split(' ')[0].replace('-','') + '&type=ALL')
    #ret = pd.read_csv(StringIO("\n".join([i.translate({ord(c): None for c in ' '})
    #                                    for i in r.text.split('\n')
    #                                    if len(i.split('",')) == 17 and i[0] != '='])), header=0)
    ret = pd.read_csv(StringIO(r.text.replace("=", "")),
            header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)
    ret = ret.set_index('證券代號')
    ret = ret[:'備註:'][:-1]
    ret = ret.loc[:,:'本益比']
    ret['成交金額'] = ret['成交金額'].str.replace(',','')
    ret['成交股數'] = ret['成交股數'].str.replace(',','')
    ret['成交筆數'] = ret['成交筆數'].str.replace(',','')
    ret['交易日']= date.date()
    return ret


  def get_twse_stocker_date_range_date(
      self,
      startdate=datetime.datetime.now(),
      n_days=10,
      sleep_time=10,
      allow_continuous_fail_count=5,
  ):
      data = {}
      date = startdate
      fail_count = 0
      while len(data) < n_days:

          print('parsing', date)
          # 使用 crawPrice 爬資料

          try:
              if pd.to_datetime(date.date()) in self.data.index.levels[0] and self.data is not None:
                print('skip! data is already having')
                data[date.date()] = self.data.loc[[date.date()]].droplevel(level='交易日')
              else:
                # 抓資料
                data[date.date()] = self.crawl_price(date)
                print('success!')
                fail_count = 0
          except:
              # 假日爬不到
              print('fail! check the date is holiday')
              fail_count += 1
              if fail_count == allow_continuous_fail_count:
                  raise
                  break

          # 減一天
          date -= datetime.timedelta(days=1)
          time.sleep(sleep_time)
      #return data
      self.data = pd.concat(data)
  def get_data(self) -> pd.DataFrame:
    self.data = pd.concat(self.get_twse_stocker_date_range_date(n_days=self.n_days))
  def save_db(self, data, db_name) -> None:
    conn = sqlite3.connect('stocker.db')  #建立資料庫
    cursor = conn.cursor()
    self.data.to_sql('Stocker', conn, if_exists='append', index=True)
  def load_db(self, db_nmae) -> None:
    conn = sqlite3.connect('stocker.db')  #建立資料庫
    cursor = conn.cursor()
    data = pd.read_sql('SELECT * FROM Stocker', conn, index_col=['交易日','證券代號'])
    data.index=data.index.set_levels(pd.to_datetime(data.index.levels[0]), level='交易日')
    data.index=data.index.set_levels(data.index.levels[1].astype(pd.StringDtype), level='證券代號')
    data[['成交股數',	'成交筆數',	'成交金額']] = data[['成交股數',	'成交筆數',	'成交金額']].astype(pd.Int64Dtype)
    self.data=data

In [ ]:
data = get_twse_stocker_date_range_date(n_days=26)

parsing 2025-09-20 13:09:14.017675
fail! check the date is holiday
parsing 2025-09-19 13:09:14.017675


/tmp/ipython-input-1054471472.py:13: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  ret = pd.read_csv(StringIO(r.text.replace("=", "")),


success!
parsing 2025-09-18 13:09:14.017675


/tmp/ipython-input-1054471472.py:13: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  ret = pd.read_csv(StringIO(r.text.replace("=", "")),


success!
parsing 2025-09-17 13:09:14.017675


...

success!
parsing 2025-09-15 13:09:14.017675


/tmp/ipython-input-1054471472.py:13: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  ret = pd.read_csv(StringIO(r.text.replace("=", "")),


success!
parsing 2025-09-14 13:09:14.017675
fail! check the date is holiday
parsing 2025-09-13 13:09:14.017675
fail! check the date is holiday
parsing 2025-09-12 13:09:14.017675


/tmp/ipython-input-1054471472.py:13: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  ret = pd.read_csv(StringIO(r.text.replace("=", "")),


success!
parsing 2025-09-11 13:09:14.017675
success!
parsing 2025-09-10 13:09:14.017675
success!
parsing 2025-09-09 13:09:14.017675
success!


In [ ]:
data[datetime.date(2025, 9, 10)].convert_dtypes().dtypes

,0
證券名稱,string[python]
成交股數,string[python]
成交筆數,string[python]
成交金額,string[python]
開盤價,string[python]
最高價,string[python]
最低價,string[python]
收盤價,string[python]
漲跌(+/-),string[python]
漲跌價差,Float64


In [ ]:
import sqlite3
conn = sqlite3.connect('stocker.db')  #建立資料庫
cursor = conn.cursor()
cursor.execute('CREATE TABLE Billionaire(Name, NetWorth, Country, Source, Rank)')  #建立資料表
conn.commit()
pd.concat(data).to_sql('Stocker', conn, if_exists='append', index=True)

298832

In [ ]:
close = pd.DataFrame({k:d['收盤價'] for k,d in data.items()}).transpose()
close.index = pd.to_datetime(close.index)
close

證券代號,1101,1101B,1102,1103,1104,1108,1109,1110,1201,1203,...,9940,9941,9941A,9942,9943,9944,9945,9946,9955,9958
2025-09-17,22.20,--,36.25,13.30,29.80,15.05,15.70,17.10,15.50,39.35,...,24.55,97.50,50.40,108.00,65.40,17.00,30.35,19.25,30.00,166.50
2025-09-16,22.25,45.45,36.25,13.35,29.85,15.05,15.75,17.70,15.55,39.60,...,24.70,97.10,50.30,109.00,65.80,17.00,30.85,19.20,29.45,169.00
2025-09-15,22.35,45.45,36.80,13.35,30.10,15.05,15.80,17.80,15.65,40.00,...,24.55,95.60,50.30,108.50,65.90,16.90,30.50,19.00,29.10,169.00
2025-09-12,22.35,45.50,36.55,13.30,31.00,15.10,15.80,17.75,15.50,39.55,...,24.50,96.50,50.30,108.50,65.60,16.90,30.55,19.15,29.45,170.50
2025-09-11,22.30,--,36.10,13.25,30.65,15.00,15.90,17.45,15.60,39.60,...,24.30,95.90,50.30,108.50,65.80,16.90,30.20,19.00,29.00,168.50
2025-09-10,22.95,--,36.40,13.45,32.00,15.20,15.80,17.45,15.60,39.90,...,24.60,96.90,50.50,109.00,66.20,17.00,31.05,19.15,29.60,171.50
2025-09-09,22.40,45.65,36.30,13.35,29.10,15.35,15.85,17.45,15.70,39.95,...,25.00,97.60,50.50,110.00,66.20,16.90,30.85,19.20,29.80,173.00
2025-09-08,22.30,45.50,36.30,13.20,29.10,15.35,15.90,17.40,15.65,39.70,...,24.70,98.60,50.50,110.50,66.20,16.95,30.80,19.10,29.10,174.00
2025-09-05,22.65,45.70,36.40,13.15,29.15,15.40,16.00,17.35,15.60,39.70,...,25.20,99.00,50.60,110.00,66.10,17.00,31.20,19.85,29.95,174.00


In [2]:
import firebase_admin
from firebase_admin import firestore

# Application Default credentials are automatically created.
app = firebase_admin.initialize_app()
db = firestore.client()

DefaultCredentialsError: Your default credentials were not found. To set up Application Default Credentials, see https://cloud.google.com/docs/authentication/external/set-up-adc for more information.

In [ ]:
class Stocker:
  def __init__(self, n_days, db_name):
    #self.name = name
    self.today = datetime.datetime.now()
    self.country = "TW"
    self.source = "TWSE"
    self.rank = ""
    self.n_days = n_days
    self.db_name = db_name

  def crawl_price(date):
    r = requests.post('http://www.twse.com.tw/exchangeReport/MI_INDEX?response=csv&date=' + str(date).split(' ')[0].replace('-','') + '&type=ALL')
    #ret = pd.read_csv(StringIO("\n".join([i.translate({ord(c): None for c in ' '})
    #                                    for i in r.text.split('\n')
    #                                    if len(i.split('",')) == 17 and i[0] != '='])), header=0)
    ret = pd.read_csv(StringIO(r.text.replace("=", "")),
            header=["證券代號" in l for l in r.text.split("\n")].index(True)-1)
    ret = ret.set_index('證券代號')
    ret = ret[:'備註:'][:-1]
    ret = ret.loc[:,:'本益比']
    ret['成交金額'] = ret['成交金額'].str.replace(',','')
    ret['成交股數'] = ret['成交股數'].str.replace(',','')
    ret['成交筆數'] = ret['成交筆數'].str.replace(',','')
    ret['交易日']= date.date()
    return ret


  def get_twse_stocker_date_range_date(
      self,
      startdate=datetime.datetime.now(),
      n_days=10,
      sleep_time=10,
      allow_continuous_fail_count=5,
  ):
      data = {}
      n_days = 9
      date = startdate
      fail_count = 0
      while len(data) < n_days:

          print('parsing', date)
          # 使用 crawPrice 爬資料
          if date.date() in self.data.index and self.data != {}:
            break
          try:
              # 抓資料
              data[date.date()] = crawl_price(date)
              print('success!')
              fail_count = 0
          except:
              # 假日爬不到
              print('fail! check the date is holiday')
              fail_count += 1
              if fail_count == allow_continuous_fail_count:
                  raise
                  break

          # 減一天
          date -= datetime.timedelta(days=1)
          time.sleep(sleep_time)
      return data
  def get_data(self) -> pd.DataFrame:
    self.data = get_twse_stocker_date_range_date(n_days=n_days)

  def save_db(self, data, db_name) -> None:
    conn = sqlite3.connect('stocker.db')  #建立資料庫
    cursor = conn.cursor()
    pd.concat(data).to_sql('Stocker', conn, if_exists='append', index=True)

  def load_db(self, db_nmae) -> None:
    conn = sqlite3.connect('stocker.db')  #建立資料庫
    cursor = conn.cursor()
    data = pd.read_sql('SELECT * FROM Stocker', conn, index_col=['交易日','證券代號'])
    data.index = data.index.set_levels(pd.to_datetime(data.index.levels[0]), level='交易日')
    self.data = data

In [12]:
Stocker2= Stocker(n_days=30,db_name="stocker.db")

In [13]:
Stocker2.load_db(Stocker2.db_name)

C:\Users\David\AppData\Local\Temp\ipykernel_7824\986836061.py:86: UserWarning: Instantiating StringDtype without any arguments.Pass a StringDtype instance to silence this warning.
  data.index=data.index.set_levels(data.index.levels[1].astype(pd.StringDtype), level='證券代號')
C:\Users\David\AppData\Local\Temp\ipykernel_7824\986836061.py:87: UserWarning: Instantiating Int64Dtype without any arguments.Pass a Int64Dtype instance to silence this warning.
  data[['成交股數',	'成交筆數',	'成交金額']] = data[['成交股數',	'成交筆數',	'成交金額']].astype(pd.Int64Dtype)


In [14]:
Stocker2.data

level_0      證券名稱      成交股數   成交筆數        成交金額     開盤價  \
交易日        證券代號                                                              
2025-09-19 0050  2025-09-19    元大台灣50  74098514  47867  4216285864   57.00   
           0051  2025-09-19   元大中型100     72633    307     6258276   86.70   
           0052  2025-09-19      富邦科技    394424   1070    88545606  225.75   
           0053  2025-09-19      元大電子     20490    139     2557407  125.40   
           0055  2025-09-19  元大MSCI金融     48522    267     1482931   30.47   
...                     ...       ...       ...    ...         ...     ...   
2025-02-19 9944        None        新麗     45553     42      927105   20.35   
           9945        None       潤泰新   2760430   1951   118600199   42.75   
           9946        None      三發地產    375817    259     9064975   23.85   
           9955        None        佳龍   1834434   1292    58672565   31.70   
           9958        None       世紀鋼   2134244   2075   374338869  172.00   

                    最高價     最低價     收盤價 漲跌(+/-)  漲跌價差  最後揭示買價 最後揭示買量  最後揭示賣價  \
交易日        證券代號                                                                
2025-09-19 0050   57.10   56.60   57.00       +  0.05   56.95    420   57.00   
           0051   86.70   85.85   85.85       -  0.30   85.80      1   85.90   
           0052  225.75  223.50  224.85       +  0.55  224.70      1  224.85   
           0053  125.40  124.25  124.40       -  0.40  124.40      1  124.50   
           0055   30.70   30.46   30.65       +  0.10   30.64      1   30.66   
...                 ...     ...     ...     ...   ...     ...    ...     ...   
2025-02-19 9944   20.35   20.30   20.35       +  0.10   20.30      1   20.35   
           9945   43.35   42.65   42.80          0.00   42.80     14   42.85   
           9946   24.40   23.85   23.85          0.00   23.85     26   23.90   
           9955   32.50   31.50   31.85       +  1.05   31.80      4   31.85   
           9958  177.50  172.00  176.00       +  4.50  175.50     27  176.00   

                最後揭示賣量     本益比  
交易日        證券代號                 
2025-09-19 0050  2,935    0.00  
           0051      3    0.00  
           0052      2    0.00  
           0053     22    0.00  
           0055      1    0.00  
...                ...     ...  
2025-02-19 9944      6  169.58  
           9945     18   10.75  
           9946     24   27.10  
           9955      4    0.00  
           9958     31   21.46  

[6475503 rows x 16 columns]

In [ ]:
Stocker2.save_db(Stocker2.data, Stocker2.db_name)

In [9]:
help(ta.momentum.kdj)

Help on function kdj in module pandas_ta.momentum.kdj:

kdj(
    high: pandas.core.series.Series,
    low: pandas.core.series.Series,
    close: pandas.core.series.Series,
    length: Union[int, numpy.integer] = None,
    signal: Union[int, numpy.integer] = None,
    offset: Union[int, numpy.integer] = None,
    **kwargs: Optional[dict]
) -> pandas.core.series.Series
    KDJ

    This indicator, derived from the Slow Stochastic, includes an
    extra signal named the J line. The J line represents the divergence
    of the %D value from the %K.

    Sources:
        * [anychart](https://docs.anychart.com/Stock_Charts/Technical_Indicators/Mathematical_Description#kdj)
        * [prorealcode](https://www.prorealcode.com/prorealtime-indicators/kdj/)

    Parameters:
        high (Series): ```high``` Series
        low (Series): ```low``` Series
        close (Series): ```close``` Series
        length (int): The period. Default: ```9```
        signal (int): Signal period. Default: ```3```

In [17]:
Stocker2.data.ta.indicators('supertrend')

Pandas TA - Technical Analysis Indicators - v0.4.71b0

Indicators and Utilities [154]:
    aberration, accbands, ad, adosc, adx, alligator, alma, alphatrend, amat, ao, aobv, apo, aroon, atr, atrts, bbands, bias, bop, brar, cci, cdl_pattern, cdl_z, cfo, cg, chandelier_exit, chop, cksp, cmf, cmo, coppock, crsi, cti, decay, decreasing, dema, dm, donchian, dpo, ebsw, efi, ema, entropy, eom, er, eri, exhc, fisher, fwma, ha, hilo, hl2, hlc3, hma, ht_trendline, hwc, hwma, ichimoku, increasing, inertia, jma, kama, kc, kdj, kst, kurtosis, kvo, linreg, log_return, long_run, macd, mad, mama, massi, mcgd, median, mfi, midpoint, midprice, mom, natr, nvi, obv, ohlc4, pdist, percent_return, pgo, pivots, ppo, psar, psl, pvi, pvo, pvol, pvr, pvt, pwma, qqe, qstick, quantile, reflex, rma, roc, rsi, rsx, rvgi, rvi, rwi, short_run, sinwma, skew, slope, sma, smc, smi, smma, squeeze, squeeze_pro, ssf, ssf3, stc, stdev, stoch, stochf, stochrsi, supertrend, swma, t3, tema, thermo, tmo, tos_stdevall, trendflex